In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install catboost
!pip3 install category_encoders
!pip install -U scikit-learn
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.0 MB/s eta 0:00:00


In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import unicodedata
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder, KBinsDiscretizer, RobustScaler, OneHotEncoder
)
from category_encoders import CountEncoder
import optuna
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder
import warnings
warnings.simplefilter("ignore")

In [2]:
# Constants
exp_name = 'exp00040'
INPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/signate2023/")
OUTPUT_PATH = f'/content/drive/MyDrive/Colab Notebooks/signate2023/{exp_name}/'
SUBMISSION_DIR = os.path.join(OUTPUT_PATH, 'submissions')
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, f'submission_{exp_name}.csv')
MODEL_DIR = os.path.join(OUTPUT_PATH, 'signate-models')
os.makedirs(MODEL_DIR, exist_ok=True)

# Load Data
train_df = pd.read_csv(INPUT_DIR / "train.csv")
test_df = pd.read_csv(INPUT_DIR / "test.csv")

In [3]:
state_dic = {}

# データ数的に、重複は気にせず上書きしていっても大した時間にはならない
for i in train_df.index:
    # ただし、欠損値で上書きされると困るので、そこだけケア
    if pd.isna(train_df.at[i, 'state'])==False:
        state_dic[train_df.at[i, 'region']] = train_df.at[i, 'state']

train_df.loc[pd.isna(train_df['state']).index, 'state'] = train_df.loc[pd.isna(train_df['state']).index, 'region'].map(state_dic)
test_df.loc[pd.isna(test_df['state']).index, 'state'] = test_df.loc[pd.isna(test_df['state']).index, 'region'].map(state_dic)

train_df.drop(['region'], axis=1, inplace=True)
test_df.drop(['region'], axis=1, inplace=True)

In [4]:
# Define a function to normalize manufacturer names
def normalize_manufacturer_name(name):
    # Convert full-width characters to half-width
    name = unicodedata.normalize('NFKC', name)
    # Convert to lowercase
    name = name.lower()
    # Remove non-ASCII characters
    name = ''.join(char for char in name if ord(char) < 128)
    return name


#fitしてtransformしないものはここで処理
def preprocessing(df):
    df[["year", "odometer"]] = df[["year", "odometer"]].astype("float32")

    df["cylinders"] = df["cylinders"].astype(str).str.extract("(\d+)").astype("float32")

    # sizeの表記揺れを修正
    df["size"] = df["size"].str.replace("ー", "-").astype(str)
    df["size"] = df["size"].str.replace("−", "-").astype(str)

    # Apply the function to the 'manufacturer' column
    df['manufacturer'] = df['manufacturer'].apply(normalize_manufacturer_name)

    df.loc[df['odometer'] == -1, 'odometer'] = np.nan


    # Define a dictionary for manufacturer name mapping with special characters
    manufacturer_name_mapping = {
        'nisan': 'nissan',
        'toyot': 'toyota',
        'nisѕan': 'nissan',
        'subαru': 'subaru',
        'subru': 'subaru',
        'sαturn': 'saturn',
        'lexu':'lexus',
        'аcura': 'acura',
        'vоlkswagen': 'volkswagen',
        'vlkswagen': 'volkswagen',
        'lexuѕ': 'lexus',
        'ᴄhrysler': 'chrysler',
        'sturn': 'saturn'
    }

    # Replace the manufacturer names using the mapping
    df['manufacturer'] = df['manufacturer'].replace(manufacturer_name_mapping)


    year_dict = {
        2999:1999,
        3008:2008,
        3011:2011,
        3015:2015,
        3017:2017,
        3019:2019,
    }
    df["year"] = df["year"].replace(year_dict)

    # 走行距離 / 製造年
    df["odometer_per_year"] = df["odometer"] / (2023 - df["year"])

    df['odometer_log'] = np.log1p(df['odometer'])

    # Create a new feature 'car_age'
    current_year = datetime.datetime.now().year
    df['car_age'] = current_year - df['year']

    # odometer_scaledとcar_ageの交互作用を新たな特徴量として作成します
    df['odometer_age_interaction'] = df['odometer'] * df['car_age']

    # 2. 'odometer'の時間経過による変化
    df['odometer_per_car_age'] = df['odometer'] / df['car_age']

    # 5. 'odometer'のビン化
    df['odometer_bin_100k'] = pd.cut(df['odometer'], bins=[0, 100000, 200000, 300000, np.inf], labels=False)

    # 6. 'odometer'の他の数学的変換
    df['odometer_sqrt'] = np.sqrt(df['odometer'])

    return df

In [5]:
train_feat_df = preprocessing(train_df)
test_feat_df = preprocessing(test_df)

In [6]:
pd.set_option('display.max_columns', None)
train_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,1556.054077,11.653982,74.0,8520952.0,1556.054077,1.0,339.334656
1,1,2013.0,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,17203.800781,12.055476,10.0,1720380.0,17203.800781,1.0,414.774628
2,2,1998.0,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,6099.680176,11.934874,25.0,3812300.0,6099.680176,1.0,390.502228
3,3,2014.0,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,11568.666992,11.553289,9.0,937062.0,11568.666992,1.0,322.673218
4,4,2005.0,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,8030.777832,11.881415,18.0,2601972.0,8030.777832,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008.0,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,1777.333374,10.190957,15.0,399900.0,1777.333374,0.0,163.278900
27528,27528,2007.0,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,5400,6754.500000,11.590562,16.0,1729152.0,6754.500000,1.0,328.743073
27529,27529,2019.0,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,34977.000000,11.848747,4.0,559632.0,34977.000000,1.0,374.042786
27530,27530,2007.0,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,7020.375000,11.629169,16.0,1797216.0,7020.375000,1.0,335.150726


In [7]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(train_feat_df, n_splits))

The variance of the mean of the folds:  4.695908800203918
0    0
1    3
2    0
3    2
4    3
5    3
6    3
7    3
8    0
9    4
Name: fold, dtype: int64
27522    4
27523    2
27524    2
27525    0
27526    0
27527    2
27528    1
27529    0
27530    1
27531    4
Name: fold, dtype: int64

In [8]:
train_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,1556.054077,11.653982,74.0,8520952.0,1556.054077,1.0,339.334656
1,1,2013.0,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,17203.800781,12.055476,10.0,1720380.0,17203.800781,1.0,414.774628
2,2,1998.0,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,6099.680176,11.934874,25.0,3812300.0,6099.680176,1.0,390.502228
3,3,2014.0,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,11568.666992,11.553289,9.0,937062.0,11568.666992,1.0,322.673218
4,4,2005.0,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,8030.777832,11.881415,18.0,2601972.0,8030.777832,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008.0,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,1777.333374,10.190957,15.0,399900.0,1777.333374,0.0,163.278900
27528,27528,2007.0,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,5400,6754.500000,11.590562,16.0,1729152.0,6754.500000,1.0,328.743073
27529,27529,2019.0,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,34977.000000,11.848747,4.0,559632.0,34977.000000,1.0,374.042786
27530,27530,2007.0,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,7020.375000,11.629169,16.0,1797216.0,7020.375000,1.0,335.150726


In [9]:
test_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,27532,2015.0,chevrolet,excellent,4.0,gas,92553.0,clean,automatic,fwd,full-size,SUV,red,co,11569.125000,11.435548,8.0,740424.0,11569.125000,0.0,304.225250
1,27533,2013.0,nissan,like new,4.0,gas,134385.0,salvage,automatic,fwd,mid-size,sedan,black,or,13438.500000,11.808472,10.0,1343850.0,13438.500000,1.0,366.585602
2,27534,2011.0,volkswagen,good,4.0,gas,102489.0,clean,automatic,fwd,full-size,sedan,black,oh,8540.750000,11.537520,12.0,1229868.0,8540.750000,1.0,320.139038
3,27535,2016.0,jeep,excellent,6.0,diesel,64310.0,clean,automatic,4wd,mid-size,SUV,red,co,9187.142578,11.071486,7.0,450170.0,9187.142578,0.0,253.594162
4,27536,1999.0,honda,excellent,8.0,gas,180839.0,rebuilt,automatic,4wd,mid-size,SUV,silver,nc,7534.958496,12.105368,24.0,4340136.0,7534.958496,1.0,425.251678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,2016.0,gmc,excellent,6.0,gas,90902.0,rebuilt,automatic,fwd,full-size,truck,black,mt,12986.000000,11.417548,7.0,636314.0,12986.000000,0.0,301.499573
27533,55065,2012.0,nissan,excellent,4.0,gas,27234.0,rebuilt,automatic,fwd,mid-size,sedan,white,nv,2475.818115,10.212258,11.0,299574.0,2475.818115,0.0,165.027267
27534,55066,2002.0,bmw,excellent,6.0,gas,99761.0,clean,automatic,rwd,full-size,coupe,blue,az,4750.523926,11.510543,21.0,2094981.0,4750.523926,0.0,315.849640
27535,55067,2006.0,bmw,excellent,6.0,gas,162279.0,clean,automatic,4wd,mid-size,sedan,silver,co,9545.823242,11.997079,17.0,2758743.0,9545.823242,1.0,402.838684


In [10]:
def preprocessing_per_fold(X_tr, y_tr, X_val, transformed_test_data, te_cols):
    scaler = RobustScaler()
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
    imputer = SimpleImputer(strategy='median')

    # RobustScaler
    X_tr['odometer_scaled'] = scaler.fit_transform(X_tr[['odometer']])
    X_val['odometer_scaled'] = scaler.transform(X_val[['odometer']])
    transformed_test_data['odometer_scaled'] = scaler.transform(transformed_test_data[['odometer']])

    # Binning
    X_tr['odometer_scaled_bin'] = pd.cut(X_tr['odometer'], bins=10, labels=False)
    X_val['odometer_scaled_bin'] = pd.cut(X_val['odometer'], bins=10, labels=False)
    transformed_test_data['odometer_scaled_bin'] = pd.cut(transformed_test_data['odometer'], bins=10, labels=False)

    X_tr['car_age_bin'] = pd.cut(X_tr['car_age'], bins=10, labels=False)
    X_val['car_age_bin'] = pd.cut(X_val['car_age'], bins=10, labels=False)
    transformed_test_data['car_age_bin'] = pd.cut(transformed_test_data['car_age'], bins=10, labels=False)

    # One-Hot Encoding for binned features
    ohe.fit(X_tr[['odometer_scaled_bin', 'car_age_bin']])
    feature_names = ohe.get_feature_names_out(['odometer_scaled_bin', 'car_age_bin'])
    columns_ohe = [name for name in feature_names]

    # Transform train, valid and test data
    X_ohe_tr = ohe.transform(X_tr[['odometer_scaled_bin', 'car_age_bin']])
    X_ohe_val = ohe.transform(X_val[['odometer_scaled_bin', 'car_age_bin']])
    test_ohe = ohe.transform(transformed_test_data[['odometer_scaled_bin', 'car_age_bin']])

    # Add the OHE columns to the dataframes
    X_tr = pd.concat([X_tr, pd.DataFrame(X_ohe_tr, columns=columns_ohe, index=X_tr.index)], axis=1)
    X_val = pd.concat([X_val, pd.DataFrame(X_ohe_val, columns=columns_ohe, index=X_val.index)], axis=1)
    transformed_test_data = pd.concat([transformed_test_data, pd.DataFrame(test_ohe, columns=columns_ohe, index=transformed_test_data.index)], axis=1)

    # Drop the original binned columns
    X_tr.drop(['odometer_scaled_bin', 'car_age_bin'], axis=1, inplace=True)
    X_val.drop(['odometer_scaled_bin', 'car_age_bin'], axis=1, inplace=True)
    transformed_test_data.drop(['odometer_scaled_bin', 'car_age_bin'], axis=1, inplace=True)


    # One-Hot Encoding for categorical columns
    onehot_cols = ['manufacturer', 'condition', 'fuel', 'title_status',
            'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
    ohe.fit(X_tr[onehot_cols])
    columns_ohe = ohe.get_feature_names_out(onehot_cols)

    # Transform train, valid and test data
    X_ohe_tr = ohe.transform(X_tr[onehot_cols])
    X_ohe_val = ohe.transform(X_val[onehot_cols])
    test_ohe = ohe.transform(transformed_test_data[onehot_cols])

    # Add the OHE columns to the dataframes
    X_tr = pd.concat([X_tr, pd.DataFrame(X_ohe_tr, columns=columns_ohe, index=X_tr.index)], axis=1)
    X_val = pd.concat([X_val, pd.DataFrame(X_ohe_val, columns=columns_ohe, index=X_val.index)], axis=1)
    transformed_test_data = pd.concat([transformed_test_data, pd.DataFrame(test_ohe, columns=columns_ohe, index=transformed_test_data.index)], axis=1)


    #クラスタリング
    # SimpleImputerをインスタンス化（ここでは欠損値を中央値で補完）
    imputer = SimpleImputer(strategy='median')
    # 補完を行いたい特徴量を選択
    features = X_tr[['odometer', 'car_age']].copy()
    # 欠損値補完
    features_imputed = imputer.fit_transform(features)
    # KMeansクラスタリングを設定
    kmeans = KMeans(n_clusters=10, random_state=0)
    # 補完されたデータに基づいてクラスタリング
    X_tr['odometer_age_cluster'] = kmeans.fit_predict(features_imputed)
    # 検証データについても同様に処理
    features_val = X_val[['odometer', 'car_age']].copy()
    features_val_imputed = imputer.transform(features_val)
    X_val['odometer_age_cluster'] = kmeans.predict(features_val_imputed)
    # テストデータについても同様に処理
    features_test = transformed_test_data[['odometer', 'car_age']].copy()
    features_test_imputed = imputer.transform(features_test)
    transformed_test_data['odometer_age_cluster'] = kmeans.predict(features_test_imputed)
    # 元のodometer_scaledの欠損値を復元
    X_tr['odometer'] = features['odometer']
    X_val['odometer'] = features_val['odometer']
    transformed_test_data['odometer'] = features_test['odometer']


    # 1. odometerに対する他の特徴量の比率
    mean_odometer_per_year = X_tr.groupby('year')['odometer'].transform('mean')
    X_tr['odometer_ratio_year'] = X_tr['odometer'] / mean_odometer_per_year
    X_val['odometer_ratio_year'] = X_val['odometer'] / mean_odometer_per_year
    transformed_test_data['odometer_ratio_year'] = transformed_test_data['odometer'] / mean_odometer_per_year

    # Count encoding
    count_cols = ["cylinders", "manufacturer", "condition", "fuel", "title_status", "transmission", "drive", "size", "type", "paint_color", "state",]
    for col in count_cols:
        count_map = X_tr[col].value_counts().to_dict()
        X_tr[col + '_count'] = X_tr[col].map(count_map)
        X_val[col + '_count'] = X_val[col].map(count_map)
        transformed_test_data[col + '_count'] = transformed_test_data[col].map(count_map)

    # Aggregate features
    agg_cols = ["manufacturer", "cylinders", "condition", "fuel", "drive", "size", "type"]

    for col in agg_cols:
        for agg_func in ['mean', 'std']:
            agg_map = X_tr.groupby(col)['odometer'].agg(agg_func).to_dict()
            X_tr[col + '_odometer_' + agg_func] = X_tr[col].map(agg_map)
            X_val[col + '_odometer_' + agg_func] = X_val[col].map(agg_map)
            transformed_test_data[col + '_odometer_' + agg_func] = transformed_test_data[col].map(agg_map)

    # Target encoding
    for col in te_cols:
        te = TargetEncoder(target_type="continuous", random_state=88)
        te.fit(X_tr[[col]], y_tr)
        X_tr['te_' + col] = te.transform(X_tr[[col]])
        X_val['te_' + col] = te.transform(X_val[[col]])
        transformed_test_data['te_' + col] = te.transform(transformed_test_data[[col]])

    return X_tr, X_val, transformed_test_data


In [11]:
def fit_lgbm(X, y, test_data, cv, te_cols, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    target_encoders = []
    transformed_test_data_list = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()

        X_tr, X_val, transformed_test_data = preprocessing_per_fold(X_tr, y_tr, X_val, transformed_test_data, te_cols)

        drop_cols = ['id', 'manufacturer', 'condition', 'fuel', 'title_status',
                     'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
        X_tr.drop(drop_cols+['price'], axis=1, inplace=True)
        X_val.drop(drop_cols+['price'], axis=1, inplace=True)
        transformed_test_data.drop(drop_cols, axis=1, inplace=True)

        transformed_test_data_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, transformed_test_data_list




def tuning(train_feat_df, train_df, cv):
    def objective(trial):
        max_depth = trial.suggest_int("max_depth", 1, 10)

        params = {
            "max_depth": max_depth,
            "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
            "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
            "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.4, 1.0),
            "subsample": trial.suggest_float("subsample", 0.4, 1.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
            "max_bin": trial.suggest_int("max_bin", 32, 512),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
            "min_sum_hessian_in_leaf": trial.suggest_float('min_sum_hessian_in_leaf', 1e-5, 1e-1),
            "feature_fraction": trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            "bagging_freq": trial.suggest_int('bagging_freq', 1, 7),
            "lambda_l1": trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
            "task": "train",
            "objective": "mape",
            "boosting": "gbdt",
            "metric": 'mape',
            "importance_type": "gain",
            "seed": 88,
            "verbosity": -1
        }

        te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state"
        ]

        score, _, _, _ = fit_lgbm(
            X=train_feat_df, y=train_df["price"], test_data=test_feat_df, cv=cv, te_cols=te_cols,  categorical_cols=[], params=params, verbose=-1
        )
        return score

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)
    print("Number of finished trials:", len(study.trials))
    print("Best trial:", study.best_trial.params)
    return study.best_trial.params

In [14]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    "n_estimators": 15683,
    "learning_rate": 0.03176950913176005,
    "max_depth": 7,
    "num_leaves": 64,
    "colsample_bytree": 0.17075595640937685,
    "subsample":  0.19095496529419081,

}

te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

# training
score, oof_lgb, models_lgb, transformed_test_data_list  = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    te_cols=te_cols,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

KeyboardInterrupt: ignored

In [12]:
lgbm_params = tuning(train_feat_df, train_df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "verbosity": -1,
    }
)

te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    te_cols=te_cols,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-17 08:57:55,274] A new study created in memory with name: no-name-e2f2ea9c-9660-4083-aafa-1648ef5556fe


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[776]	valid_0's mape: 0.458529
 - fold1 - 0.4585
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[964]	valid_0's mape: 0.469177
 - fold2 - 0.4692
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[830]	valid_0's mape: 0.458976
 - fold3 - 0.4590
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[841]	valid_0's mape: 0.470391
 - fold4 - 0.4704
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[662]	valid_0's mape: 0.453215
 - fold5 - 0.4532
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[777]	valid_0's mape: 0.4565
 - fold6 - 0.4565
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[716]	valid_0's mape: 0.47170

[I 2023-08-17 08:59:52,522] Trial 0 finished with value: 0.46219137788278025 and parameters: {'max_depth': 1, 'n_estimators': 19178, 'learning_rate': 0.043121408232353366, 'num_leaves': 2, 'colsample_bytree': 0.9970176335251624, 'subsample': 0.8746866822603172, 'min_data_in_leaf': 130, 'max_bin': 355, 'reg_lambda': 77.78342061282417, 'min_sum_hessian_in_leaf': 0.019726804228419742, 'feature_fraction': 0.5821198083909772, 'bagging_fraction': 0.5584217624038312, 'bagging_freq': 4, 'lambda_l1': 5.3807862951953265e-05}. Best is trial 0 with value: 0.46219137788278025.


Early stopping, best iteration is:
[1284]	valid_0's mape: 0.459038
 - fold8 - 0.4590
FINISHI: Whole Score: 0.4622
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[182]	valid_0's mape: 0.436988
 - fold1 - 0.4370
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[180]	valid_0's mape: 0.448875
 - fold2 - 0.4489
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[90]	valid_0's mape: 0.439562
 - fold3 - 0.4396
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[149]	valid_0's mape: 0.451574
 - fold4 - 0.4516
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[118]	valid_0's mape: 0.43728
 - fold5 - 0.4373
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[171]	valid_0's mape: 0.437054
 - fold6 - 0.4371
Training unt

[I 2023-08-17 09:00:39,714] Trial 1 finished with value: 0.4430565603090438 and parameters: {'max_depth': 8, 'n_estimators': 20948, 'learning_rate': 0.08341409324266379, 'num_leaves': 31, 'colsample_bytree': 0.9633450635462286, 'subsample': 0.5254764712817813, 'min_data_in_leaf': 289, 'max_bin': 188, 'reg_lambda': 7.47017581061206, 'min_sum_hessian_in_leaf': 0.07605206958226675, 'feature_fraction': 0.6916715883834158, 'bagging_fraction': 0.7150916645015777, 'bagging_freq': 3, 'lambda_l1': 0.0013726928267069225}. Best is trial 1 with value: 0.4430565603090438.


...

FINISHI: Whole Score: 0.4419
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[309]	valid_0's mape: 0.436478
 - fold1 - 0.4365
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[191]	valid_0's mape: 0.447468
 - fold2 - 0.4475
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[112]	valid_0's mape: 0.438957
 - fold3 - 0.4390
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[377]	valid_0's mape: 0.452521
 - fold4 - 0.4525
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[356]	valid_0's mape: 0.43638
 - fold5 - 0.4364
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[323]	valid_0's mape: 0.434143
 - fold6 - 0.4341
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:

[I 2023-08-17 10:46:33,103] Trial 98 finished with value: 0.442166342414447 and parameters: {'max_depth': 7, 'n_estimators': 36712, 'learning_rate': 0.046571960863992914, 'num_leaves': 68, 'colsample_bytree': 0.8433787772011718, 'subsample': 0.5286396073776345, 'min_data_in_leaf': 293, 'max_bin': 84, 'reg_lambda': 53.53872740525331, 'min_sum_hessian_in_leaf': 0.09725856175620476, 'feature_fraction': 0.8767858359018296, 'bagging_fraction': 0.8893653502923774, 'bagging_freq': 6, 'lambda_l1': 2.516967212235949e-05}. Best is trial 71 with value: 0.44150416227201267.


Early stopping, best iteration is:
[321]	valid_0's mape: 0.441313
 - fold8 - 0.4413
FINISHI: Whole Score: 0.4422
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[333]	valid_0's mape: 0.436531
 - fold1 - 0.4365
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[344]	valid_0's mape: 0.447254
 - fold2 - 0.4473
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[196]	valid_0's mape: 0.43852
 - fold3 - 0.4385
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[280]	valid_0's mape: 0.450477
 - fold4 - 0.4505
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[284]	valid_0's mape: 0.43724
 - fold5 - 0.4372
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[439]	valid_0's mape: 0.435841
 - fold6 - 0.4358
Training unti

[I 2023-08-17 10:47:41,216] Trial 99 finished with value: 0.44211845150093493 and parameters: {'max_depth': 6, 'n_estimators': 34967, 'learning_rate': 0.04204565295201745, 'num_leaves': 39, 'colsample_bytree': 0.8866440862857925, 'subsample': 0.5775820003668345, 'min_data_in_leaf': 285, 'max_bin': 107, 'reg_lambda': 49.970333021285924, 'min_sum_hessian_in_leaf': 0.09182490649814219, 'feature_fraction': 0.9125355334779053, 'bagging_fraction': 0.9223974253022242, 'bagging_freq': 4, 'lambda_l1': 3.068223356709773e-05}. Best is trial 71 with value: 0.44150416227201267.


Early stopping, best iteration is:
[448]	valid_0's mape: 0.441197
 - fold8 - 0.4412
FINISHI: Whole Score: 0.4421
Number of finished trials: 100
Best trial: {'max_depth': 9, 'n_estimators': 40291, 'learning_rate': 0.04931580470059597, 'num_leaves': 336, 'colsample_bytree': 0.8751115136419975, 'subsample': 0.5161411427739584, 'min_data_in_leaf': 262, 'max_bin': 80, 'reg_lambda': 44.60370154255466, 'min_sum_hessian_in_leaf': 0.08518046469002956, 'feature_fraction': 0.8619950259400452, 'bagging_fraction': 0.9757754475325312, 'bagging_freq': 5, 'lambda_l1': 0.00018825283207066316}
 - fold1 - 0.4373
 - fold2 - 0.4464
 - fold3 - 0.4389
 - fold4 - 0.4511
 - fold5 - 0.4355
 - fold6 - 0.4327
 - fold7 - 0.4493
 - fold8 - 0.4411
FINISHI: Whole Score: 0.4415


In [13]:
# Create a copy of the dataframe
#train_data_transformed = train_feat_df.copy()#

# Apply target encoding to the copy of the dataframe
#for col in te_cols:
#    # Create a new instance of TargetEncoder and fit it to the whole data
#    te = TargetEncoder(target_type="continuous", random_state=88)
#    te.fit(train_data_transformed[[col]], train_df["price"])
#    train_data_transformed['te_' + col] = te.transform(train_data_transformed[[col]])

## Drop the original columns
#drop_cols = ['ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
#       'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state']
#train_data_transformed.drop(drop_cols, axis=1, inplace=True)

## Check the feature importances
#importances = pd.DataFrame({
#    'Feature': train_data_transformed.columns,
##    'Importance': models_lgb[0].feature_importances_
#}).sort_values(by='Importance', ascending=False)
#importances


In [14]:
sub_df = pd.read_csv(INPUT_DIR / "submit_sample.csv", names=["id", "price"])

In [15]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'{MODEL_DIR}/submission.csv', index=False, header=False)


In [16]:
pd.DataFrame(oof_lgb, columns=['oof_pred']).to_csv(f'{MODEL_DIR}/exp40_oof_pred.csv', index=False)